In [2]:
import sqlalchemy
import pandas as pd
from db_engine import DBEngine

In [3]:
user = 'h.zand'
password = '951c898a1f9af97acec3fb9b8df4'
host = '172.30.5.214'
port = '13306'
database = 'digikala_log'
assert user != "" and password != "", "please enter your user and password"
log_engine = sqlalchemy.create_engine(
    f'mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}')

In [15]:
start_date = '2023-09-12'

In [16]:
carts_query = f'''
select entity_id as order_id, changed_at, changes, command
from digikala_log.carts_log
where changed_at > '{start_date}';
'''

In [17]:
cart_items_query = f'''
select entity_id as order_item_id, changed_at, changes, command
from digikala_log.carts_log
where changed_at > '{start_date}';
'''

In [7]:
carts_log = pd.read_sql(carts_query, DBEngine.get_digikala_log_engine())
cart_items_log = pd.read_sql(cart_items_query, DBEngine.get_digikala_log_engine())

In [17]:
carts_log['entity'] = 'cart'
cart_items_log['entity'] = 'cart_item'

In [11]:
cart_items_log

In [12]:
query = f'''
select ci.id as cart_item_id,
       c.id  as cart_id,
       c.user_id,
       c.status,
       c.payable_price,
       c.source,
       c.source_close
from cart_items ci
         join carts c on c.id = ci.cart_id
where c.updated_at > '{start_date}'
-- where (ci.updated_at > '{start_date}' or c.updated_at > '{start_date}')
  and c.site = 'digikala'
  and c.business_type = 'b2c'
  and c.source != 'core'
  and c.source != 'mehr';
'''

In [13]:
cart_and_cart_items = pd.read_sql(query, DBEngine.get_digikala_engine())

,cart_item_id,cart_id,user_id,status,payable_price,source,source_close
0,2792752997,350441040,NaN,open,164800,android,None
1,2792753022,350441041,11735509.0,open,0,android,None
2,2792753288,350441041,11735509.0,open,0,android,None
3,2792767070,350441041,11735509.0,open,0,android,None
4,2792753024,350441042,59419178.0,open,5340000,web,None
...,...,...,...,...,...,...,...
219790,2793380242,350489875,13440623.0,open,0,mobile,None
219791,2793380261,350489876,18890777.0,open,0,android,None
219792,2793380266,350489877,17036976.0,open,0,mobile,None
219793,2793380279,350489878,NaN,open,0,web,None


,order_id,changed_at,changes,command,entity
0,349864676,2023-09-12 00:00:01,"{""payable_price"":981000,""vat"":80999}",/v1/cart/remove/,cart
1,345901453,2023-09-12 00:00:01,"{""payable_price"":27755000,""vat"":149448}",/v1/cart/,cart
2,325141039,2023-09-12 00:00:01,"{""payable_price"":17880000}",/v1/cart/,cart
3,283599921,2023-09-12 00:00:01,"{""payable_price"":4190000,""vat"":0}",/v1/cart/,cart
4,350253760,2023-09-12 00:00:01,"{""shipping_cost_total"":800000,""shipping_cost"":...",/v1/shipment/,cart
...,...,...,...,...,...
841644,348629959,2023-09-12 11:46:17,"{""payable_price"":2391000}",/v1/cart/,cart
841645,343567069,2023-09-12 11:46:17,"{""payable_price"":3980000,""vat"":0,""shipping_cos...",/v1/cart/,cart
841646,350488551,2023-09-12 11:46:17,"{""shipping_cost_vat"":66055,""shipping_cost"":733...",/v1/payment/,cart
841647,350058171,2023-09-12 11:46:17,"{""discount"":200000,""payable_price"":470000,""vat...",/v1/cart/remove/,cart
